Display the different models performance

In [ ]:
import os 
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

custom_params = {"axes.spines.right": False, "axes.spines.top": False, "axes.spines.left": False,
                 "axes.spines.bottom": False, "figure.dpi": 100}
sns.set_theme(style = "whitegrid", rc = custom_params, font_scale = 1.75)

In [ ]:
path = 'results/'

In [ ]:
simulations, all = {}, {}
names = {'indi': 'Individuals', 'poin': 'Points', 'time': 'Time', 'miss': 'Misspecification', 'rand': 'Random effect', 'corr': 'Correlated Random Effect', 'noco': 'Uncorrelated Data', 'sinu': 'Sinus'}
for file_name in os.listdir(path):
    name = file_name[:file_name.rindex('.')].replace('_summary', '')
    exp = names[name[:4]]
    try:
        value = float(name.replace('individuals', '').replace('points', ''))
    except:
        value = np.nan
    with open(path + file_name) as json_file:
        if 'summary' in file_name:
            simulations[(exp, value)] = json.load(json_file)
        else:
            all[(exp, value)] = json.load(json_file)
            print(exp, value, all[(exp, value)]['start'][0])
            n_experiments = all[(exp, value)]['start'][0] - 1

# Displays error

In [ ]:
models = {'correct': 'Correct', 
          'mm': 'MM',
          'all': 'All', 
          'outcomes': r'Mis. $y$', 
          'sigma': r'Mis. $\omega$', 
          'nore': r'No $u^\omega$',
          'sigmanore': r'No $u^\omega$ + Mis. $\omega$',
          'melsm_notime': 'No Time',
          'gaussian': r'Gaussian $u^\omega$', 
          'non_sinus': 'Non sinus',
          'incorrect': r'Uncorrelated ($u^\omega$, $u^y$)'}

covariates = {'time': 'Time', 
             'beta': r'$\beta^y$',
             'beta_out_error': r'$\beta^y$',
             'beta_out_relative': r'$\beta^y$',
             'beta_out_coverage': r'$\beta^y$',
             'beta_omega': r'$\beta^{\omega}$',
             'beta_omega_error': r'$\beta^{\omega}$',
             'beta_omega_relative': r'$\beta^{\omega}$',
             'beta_omega_coverage': r'$\beta^{\omega}$',
             're_out': r'$u^y$',
             're_omega_0': r'$u^{\omega}_0$',
             're_omega_1': r'$u^{\omega}_1$',
             'sd_re_out': r'$\sigma_{y}$',
             'sd_re_omega_0': r'$\sigma_{\omega, 0}$',
             'sd_re_omega_1': r'$\sigma_{\omega, 1}$',
             'last': 'Outcome',
             'average': 'Outcome'}

cov_names = ['Age', 'Albumin', 'Trig', 'Platelet']

In [ ]:
parameter = 'beta_out_error' 
dimension = 1

In [ ]:
compare, error, all_selection = {}, {}, {}
for simulation in simulations:
    compare[simulation], error[simulation] = {}, {}
    for model in simulations[simulation]:
        if parameter in simulations[simulation][model]:
            compare[simulation][models[model]] = simulations[simulation][model][parameter]['value'][dimension if dimension is not None else 0]
            error[simulation][models[model]] = simulations[simulation][model][parameter]['std'][dimension if dimension is not None else 0]
            all_selection[tuple(list(simulation) + [models[model]])] = np.array(all[simulation][model][parameter])[:, dimension] if dimension is not None else np.array(all[simulation][model][parameter])[:,0]
            
    compare[simulation], error[simulation] = pd.Series(compare[simulation]), pd.Series(error[simulation])
compare, error = pd.concat(compare, axis = 1), pd.concat(error, axis = 1)
compare, error = compare.T.sort_index().T.loc[[m for m in models.values() if m in compare.index]], error.T.sort_index().T.loc[[m for m in models.values() if m in compare.index]]
all_selection = pd.DataFrame.from_dict(all_selection)

In [ ]:
for exp in names.values():
    if parameter == 'time':
       xlabel = 'Relative Time'
       ratio = compare.loc[:, exp].dropna().rename(models)
       ratio = ratio / ratio.loc['correct']
       ratio[ratio.index != 'correct'].plot.barh(legend = False)
    else:
        try:
            ax = compare.loc[:, exp].dropna().rename(models).plot.barh(legend = False, xerr = 1.96 * error.loc[:,exp].dropna() / np.sqrt(n_experiments))
            plt.xlim(max(np.min(compare.loc[:, exp].dropna())- 0.5 * np.std(compare.loc[:, exp].dropna().values), 0), None)
            if 'coverage' in parameter:
                xlabel = 'Coverage (%)'
            else:
                xlabel = 'Absolute error'
        except:
            continue
    plt.xlabel(xlabel)
    plt.ylabel('Models')
    plt.title('{}[{}] - {} experiment'.format(covariates[parameter], cov_names[dimension], exp) if dimension is not None else '{} - {} experiment'.format(covariates[parameter], exp))
    if len(compare.loc[:, exp].columns) > 1:
        plt.legend(title = exp, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(alpha = 0.3)

    plt.savefig('images/{}/{}_{}.png'.format(exp, parameter, dimension), bbox_inches='tight')

In [ ]:
true_values = {
    "beta": [0.5, 0.5, 0, 0],
    "beta_omega": [0.8, 0., 0.8, 0.],
}

In [ ]:
assert ((parameter == "beta_omega") or (parameter == "beta")), "Display distribution only for parameters estimates"

for exp in names.values():
      try: 
            param_name = '{}[{}]'.format(covariates[parameter], cov_names[dimension])
            data = all_selection[exp].dropna().stack([0, 1]).reset_index(level=0, drop=True).to_frame(name = param_name).reset_index(names = [exp, "Models"])
            data = data.dropna(axis = 'columns', how = 'all')
            sns.boxplot(data, x = param_name, y = "Models", order = [m for m in models.values() if m in data.Models.unique()][::-1], hue = exp if exp in data.columns else None, color='tab:blue'if exp not in data.columns else None)

            plt.axvline(true_values[parameter][dimension], color = 'tab:red', linestyle = '--')
            plt.title(exp)
            if len(compare.loc[:, exp].columns) > 1:
                  plt.legend(title = exp, loc='center left', bbox_to_anchor=(1, 0.5))
            plt.savefig('images/{}/{}_{}.png'.format(exp, parameter, dimension), bbox_inches='tight')
            plt.show()
      except Exception as e:
            print(e)
